In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()


Workspace name: quick-starts-ws-189736
Azure region: southcentralus
Subscription id: 1b944a9b-fdae-4f97-aeb1-b7eea0beac53
Resource group: aml-quickstarts-189736


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "mlcluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)



InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
#ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling( {
    "--C": uniform(0.5, 50), 
    "--max_iter": choice(75, 100, 125, 150)
    }
)

# Specify a Policy
#policy = ### YOUR CODE HERE ###
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
#src = ### YOUR CODE HERE ###
#from azureml.core import ScriptRunConfig, Experiment

#    # create or load an experiment
#    experiment = Experiment(workspace, 'MyExperiment')
#    # create or retrieve a compute target
#    cluster = workspace.compute_targets['MyCluster']
#    # create or retrieve an environment
#    env = Environment.get(ws, name='MyEnvironment')
   # configure and submit your training run
src = ScriptRunConfig(source_directory='.',
                            script='train.py',
                            #arguments=['--C', arg1_val, '--max_iter', arg2_val],
                            compute_target=cpu_cluster,
                            environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=50,
                             max_concurrent_runs=4)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
from azureml.widgets import RunDetails

hyperdrive_run = exp.submit(hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)

RunDetails(hyperdrive_run).show()

RunId: HD_cad88e21-b630-4adf-8315-27ad008b8166
Web View: https://ml.azure.com/runs/HD_cad88e21-b630-4adf-8315-27ad008b8166?wsid=/subscriptions/1b944a9b-fdae-4f97-aeb1-b7eea0beac53/resourcegroups/aml-quickstarts-189736/workspaces/quick-starts-ws-189736&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-03-23T07:03:01.190492][API][INFO]Experiment created<END>\n""<START>[2022-03-23T07:03:02.047286][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-03-23T07:03:02.688340][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_cad88e21-b630-4adf-8315-27ad008b8166
Web View: https://ml.azure.com/runs/HD_cad88e21-b630-4adf-8315-27ad008b8166?wsid=/subscriptions/1b944a9b-fdae-4f97-aeb1-b7eea0beac53/resourcegroups/aml-quickstarts-189736/workspaces/quick-starts-ws-189736&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [25]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])
print("Parameter " + parameter_values[0] + " : ",parameter_values[1])
print("Parameter " + parameter_values[2] + " : ",parameter_values[3])

print("Best Run Metrics:",best_run.get_metrics())
print("Best run file names :",best_run.get_file_names())

# model = best_run.register_model(model_name='hyperdrive-best-model', model_path='outputs/model.joblib')

Best Run Id:  HD_cad88e21-b630-4adf-8315-27ad008b8166_7

 Accuracy: 0.9104151493080845
['--C', '36.274556557960295', '--max_iter', '125']
Parameter --C :  36.274556557960295
Parameter --max_iter :  125
Best Run Metrics: {'Regularization Strength:': 36.274556557960295, 'Max iterations:': 125, 'Accuracy': 0.9104151493080845}
Best Run Details Parameter: ['--C', '36.274556557960295', '--max_iter', '125']
Best run file names : ['logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'user_logs/std_log.txt']


In [ ]:
model_name = best_run.properties["model_name"]
print(model_name)

In [ ]:
model = best_run.register_model(model_name='hyperdrive-best-model',
                           description='Best Run for the Hyperdrive experiment', tags=None)

In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=url_path)

In [10]:
pdf = ds.to_pandas_dataframe()
pdf.head(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


In [11]:
pdf.shape

(32950, 21)

In [14]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
# x and y are dataframes
x, y = clean_data(ds)
df = pd.concat([x, y], axis=1)

# train_data, test_data = train_test_split(df, test_size=0.25, random_state=50)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=50)


In [15]:
df.head(5)

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


In [16]:
train_data, test_data = train_test_split(df, test_size=0.25, random_state=50)

In [17]:
train_data.head(5)

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
8125,30,1,0,0,0,6,1,108,1,999,...,0,0,0,0,0,0,0,1,0,0
4837,45,0,0,1,0,5,1,8,7,999,...,0,0,0,0,1,0,0,0,0,0
31840,48,1,0,1,0,8,4,602,3,999,...,0,0,0,1,0,0,0,0,0,0
15054,43,1,0,0,0,11,4,72,2,999,...,0,0,0,1,0,0,0,0,0,0
13917,33,0,0,0,1,5,5,128,1,999,...,1,0,0,0,0,0,0,1,0,0


In [18]:
test_data.head(5)

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
1314,31,1,0,0,0,5,3,81,1,999,...,0,1,0,0,0,0,0,0,0,0
13350,33,0,0,1,0,4,4,230,1,999,...,0,0,0,0,0,0,0,1,0,0
8446,28,1,0,1,0,5,1,643,1,999,...,0,0,0,0,0,0,0,1,0,1
23907,32,0,0,1,0,11,1,128,2,999,...,0,0,0,0,1,0,0,0,0,0
5400,28,0,0,1,1,5,2,6,5,999,...,0,0,0,0,1,0,0,0,0,0


In [19]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,  
    label_column_name='y',
    n_cross_validations=2)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
from azureml.widgets import RunDetails


automl_run = exp.submit(automl_config)
automl_run.wait_for_completion(show_output=True)

RunDetails(automl_run).show()

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = automl_run.get_output()

#best_run = automl_run.get_best_child()
print(best_run)
print("Best run metrics :",best_run.get_metrics())
print("Best run details :",best_run.get_details())
print(fitted_model.steps)

# model_name = best_run.properties['model_name']
# description = 'AutoML classification Best Run'
# tags = None

model = best_run.register_model(model_name = best_run.properties['model_name'], 
                                  description = 'AutoML classification Best Run', 
                                  tags = None)

###########################################                                  
# automl_best_run, automl_best_model = automl_run.get_output()

# print("Best run metrics :", automl_best_run)
# # print("Best run details :",automl_run.get_details())
# # print("Best run file names :",best_run.get_file_names())

# best_automl_model = automl_run.register_model(model_name='best_automl_model')
